# OpenFE Showcase: Relative Binding Free Energy of T4 Lysozyme


## 0. Setup for Google Colab

If you are running this example in Google Colab, run the following cells to setup the environment. If you are running this notebook locally, skip down to `1. Overview`

In [ ]:
# NBVAL_SKIP
# Only run this cell if on google colab
import os
if "COLAB_RELEASE_TAG" in os.environ:
    # fix for colab's torchvision causing issues
    !rm -r /usr/local/lib/python3.12/dist-packages/torchvision
    
    !pip install -q condacolab
    import condacolab
    condacolab.install_from_url("https://github.com/OpenFreeEnergy/openfe/releases/download/v1.7.0/OpenFEforge-1.7.0-Linux-x86_64.sh")

In [ ]:
# NBVAL_SKIP
# Only run this cell if on google colab
import os
if "COLAB_RELEASE_TAG" in os.environ:
    import condacolab
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"
    !mkdir inputs && cd inputs && openfe fetch rbfe-tutorial
    # quick fix for https://github.com/conda-incubator/condacolab/issues/75
    # if deprecation warnings persist, rerun this cell
    import warnings
    warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow") 
    for _ in range(3):
      # Sometimes we have to re-run the check
      try:
        condacolab.check()
      except:
        pass
      else:
        break

## 1. Overview

In this example we show how to set up a network of transformations using
the OpenFE toolkit for small chemical modifications of ligands binding to tyrosine kinase 2 (TYK2).

This notebook demonstrates the current capabilities of the OpenFE toolkit. Here
we specifically look at how relative binding free energy calculations can be
carried out using the OpenMM MD engine in combination with the OpenFE toolset.
This notebook will be continually updated as OpenFE introduces new features.

For convenience, a prepared (capped and protonated) PDB structure of the
TYK2 protein is provided under `inputs/tyk2_protein.pdb`.

<img src="attachment:da4bf032-9a72-4e2d-9e6d-58e1ac5face9.png" width="300">

### Alchemical transformations of TYK2 ligands

Here we explore how OpenFE can be used
to build a simple star map network for planned alchemical substitutions of the TYK2 ligands.

Ten pre-aligned compounds are available under
`inputs/tyk2_ligands.sdf`. These are shown in the cell below.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

# Extract the contents of the sdf file and visualise it
ligands_rdmol = [mol for mol in
                 Chem.SDMolSupplier('inputs/tyk2_ligands.sdf', removeHs=False)]

for ligand in ligands_rdmol:
    AllChem.Compute2DCoords(ligand)

Chem.Draw.MolsToGridImage(ligands_rdmol)

Ultimately we will limit the RBFEs done in this notebook to the transformation
of `lig_ejm_31` to `lig_ejm_47`. 

### The plan

Here is what we will achieve in this notebook and what software toolchains are
used along the way.

| **Actions**                    | **Software**                                                |
|:------------------------------:|:-----------------------------------------------------------:|
| Create OpenFE Molecules        | OpenFE tk, RDKit                                            |
| Create Radial Network          | OpenFE tk, Lomap, Networkx                                  |
| Visualise Network              | OpenFE tk, NetworkX, RDKit, Matplotlib                      |
| Create ligand topologies       | OpenFE tk interface - OpenFF tk                             |
| Create hybrid OpenMM topology  | OpenFE tk interface - OpenMMTools (eventually - ex Perses)  |
| Create Lambda Protocol         | OpenFE tk interface - OpenMMTools (eventually - ex Perses)  |
| Setup and run RBFE calculation | OpenFE tk interface - OpenMM + OpenMMTools                  |
| Analysis RBFE calculation      | OpenFE tk interface - PyMBAR + OpenMMTools                  |


A complete overview of the setup and simulation process starting from initial SDF and PDB files can be found [in this diagram](./assets/RBFE_workflow.drawio.pdf).

## 2. Creating a transformation network

### Creating OpenFE SmallMoleculeComponents

In order to keep track of the various inputs being passed through the OpenFE
toolkit, OpenFE implements a set of Components which define the proteins,
small molecules and solvent components which a system may contain. Here we
use the [SmallMoleculeComponent](https://github.com/OpenFreeEnergy/gufe/blob/main/gufe/smallmoleculecomponent.py)
which takes in either [RDKit molecules](https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html)
or [OpenFF molecules](https://open-forcefield-toolkit.readthedocs.io/en/0.9.2/api/generated/openff.toolkit.topology.Molecule.html).

In the backend, OpenFE treats the RDKit molecules as the central representation
of the ligands, and uses the OpenFF toolkit to convert between objects from
various toolchains (for example OpenEye's OEMol).

Here we demonstrate how to load the ligands from `inputs/tyk2_ligands.sdf` into a
list of OpenFE SmallMoleculeComponents for further processing.

In [ ]:
import locale
locale.getpreferredencoding = lambda _: 'UTF-8'  # hack for google colab, not needed for local execution
from openfe import SmallMoleculeComponent

# Create and SDF supplier
# Extract the contents of the sdf file and visualise it
ligands_sdf = Chem.SDMolSupplier('inputs/tyk2_ligands.sdf', removeHs=False)

# Now pass these to form a list of Molecules
ligand_mols = [SmallMoleculeComponent(sdf) for sdf in ligands_sdf]

OpenFE SmallMoleculeComponents have some useful built in attributes and methods.

For example the molecule's name (as defined by the SDF file) can be accessed

In [ ]:
print("name: ", ligand_mols[0].name)

In [ ]:
ligand_mols[0].to_dict()

As previously stated SmallMoleculeComponents also use the OpenFF backend to allow conversion between different object types. For example it's possible to obtain an openff Molecule:

In [ ]:
type(ligand_mols[0].to_openff())

### Creating a network

From these SmallMoleculeComponents we can quickly create a star map network which centers
around the first ligand in our sdf file using
`openfe.setup.ligand_network_planning.generate_radial_network`.

Here we use Lomap (via the OpenFE interface LomapAtomMapper) to define the atom mappings between the various ligands and the central ligand. Whilst
we use the defaults here, please note that the various supported arguments of
Lomap can be passed to LomapAtomMapper.

Note: LomapAtomMapper is one of three currently implemented Mappers in OpenFE, the other two being the KartografAtomMapper and the rjmc mapper from Perses. Long term, users will be able to pass serveral mappers and atom mapping scoring methods on network generation.

In [ ]:
# Create network from the two molecules
from openfe.setup.ligand_network_planning import generate_radial_network
from openfe.setup import LomapAtomMapper

network = generate_radial_network(ligands=ligand_mols[1:],
                                  central_ligand=ligand_mols[0],
                                  mappers=[LomapAtomMapper(threed=True, element_change=False),])

This network contains a set of 9 edges centered around ligand `lig_ejm_31`
which define the various atom mappings between the the transformation pairs.


Edges along the network can be accessed to recover the invidual molecules
involved in that given alchemical tranformation, and the atom mapping between
the two ligands.

**Note: as can be seen in the example below, transformations are defined within OpenFE as going from componentA to componentB**

In [ ]:
transform_edges = [edge for edge in network.edges]
print("molecule A smiles: ", transform_edges[0].componentA.smiles)
print("molecule B smiles: ", transform_edges[0].componentB.smiles)
print("map between molecule A and B: ", transform_edges[0].componentA_to_componentB)

We can also visualise the atom mappings by invoking the individual OpenFE
AtomMapping objects directly.

Unique atoms between each mapping are shown in red, and atoms which are mapped
but undergo element changes are shown in blue. Bonds which either involve atoms that are unique or undergo element changes are highlighted in red.

In [ ]:
# Looking at the mapping of an edge
edge = [edge for edge in network.edges if edge.componentB.name == "lig_ejm_47"][0]
edge

In [ ]:
from IPython.display import Image

# mappings can also be saved to file if required
edge = [edge for edge in network.edges if edge.componentB.name == "lig_ejm_47"][0]
edge.draw_to_file('tyk2_edge.png')

# load it back for visualisation
Image("tyk2_edge.png")

In [ ]:
# Display all the atom mappings
edges = [edge for edge in network.edges]
for edge in edges:
    display(edge)

### Storing the network

Created networks can easily be converted to (and also loaded from) as a GraphML representation.

This can allow users of OpenFE to store the network to disk for later use.

In [ ]:
# Convert to graphml
with open("network_store.graphml", "w") as writer:
    writer.write(network.to_graphml())

In [ ]:
from openfe.setup import LigandNetwork

# load a new network from this graphml representation
with open('network_store.graphml', 'r') as file:
    network_data = file.read()

new_network = LigandNetwork.from_graphml(network_data)

In [ ]:
edges = [edge for edge in new_network.edges]
print(f"edge 0 molecule 1: {edges[0].componentA.name}")
print(f"edge 0 molecule 2: {edges[0].componentB.name}")
print(f"edge 0 mapping: {edges[0].componentA_to_componentB}")

### The OpenFE CLI

The OpenFE toolkit also has a command line interface which we plan to add several convenience methods to.

In [ ]:
# Call help on the OpenFE CLI
locale.getpreferredencoding = lambda: 'UTF-8'  # hack for google colab, not needed for local execution
!openfe --help

In [ ]:
# Get an atom mapping using the CLI

!openfe atommapping --mapper LomapAtomMapper --mol inputs/benzene.sdf --mol inputs/styrene.sdf

In [ ]:
# We can also directly visualise the mapping
!openfe atommapping --mapper LomapAtomMapper --mol inputs/benzene.sdf --mol inputs/styrene.sdf --output test.png

Image('test.png')

## 3. Running a Relative Ligand Binding Free Energy Calculation

Now that we have a set of atom mappings defined, we know which atoms should
undergo alchemical transformations to capture the free energy cost of
transforming from one ligand to another.

To simulate this transformation we use the equilibrium RBFE protocol
implemented in OpenFE. This uses OpenMM to run a Perses-like relative
ligand binding free energy calculation using a single topology approach.

To achieve this simulation, the following steps need to happen:

1. Create OpenMM systems of both end states


2. Create a hybrid topology based on these defined endstates


3. Set an appropriate Lambda schedule


4. Set a MultiState reporter to write out appropriate coordinates and energies


5. Create an OpenMM sampler (in this case we will be using a replica exchange sampler)


6. Carry out the necessary simulation steps (minimization, equilibration, and production)


The `RelativeHybridTopologyProtocol` class in `openfe.protocols.openmm_rfe`
implements a means to achieve all the above with minimal intervention.

Here we work through its usage for the `lig_ejm_31` -> `lig_ejm_47` binding free energy
test case. As this involves both a relative binding free energy in solvent
and complex phases, the `RelativeHybridTopologyProtocol` Protocol will
be used to build two separate `ProtocolDAG` (directed-acyclic-graph) classes, one for each phase.
These `DAG`s (which contain the necessary individual simulations), are then executed to yield
the desired free energy results.

**Note: the underlying components used for the creation of OpenMM hybrid
topologies and samplers is still in flux, originating mostly from Perses.
Please consider these to be in beta.**

### Defining the ChemicalSystems

`ChemicalSystems` are OpenFE containers which define the various components
which exist in a system of interest. You can consider these to be the nodes
along an alchemical network which are connected by edges which carry out
calculations along Alchemical states to get free energies.

`ChemicalSystems` take in three different things:

1. A dictionary of the chemical components (e.g. `SmallMoleculeComponent`,
   `ProteinComponent`, `SolventComponent`) defining the system.


2. Box vectors (optional), defining the shape and size of the unit cell of the
   system.


3. An identifier name (optional), for the `ChemicalSystem`. This is used as part
   of the hash identifier of the `ChemicalSystem`, and can help distinguish between
   otherwise comparable systems.

In the case of a relative ligand binding free energy calculation for `lig_ejm_31` -> `lig_ejm_47`,
four `ChemicalSystems` must be defined:

1. `lig_ejm_31` in complex with TYK2 in a box of water


2. `lig_ejm_47` in complex with TYK2 in a box of water


3. `lig_ejm_31` in a box of water


4. `lig_ejm_47` in a box of water


Here we will be passing the previously defined `SmallMoleculeComponents` for `lig_ejm_31`
and `lig_ejm_47`. We will also pass a `ProteinComponent` generated from the PDB file
present under `inputs/tyk2_protein.pdb`. Finally, instead of passing
in a specific box of water, we will define a `SolventComponent` which will contain
the necessary information for OpenMM's `Modeller` class to add water and 0.15 M NaCl
around the solute when creating the OpenMM simulation objects.

In [ ]:
# First let's define the Protein and Solvent Components which we will be using
from openfe import SolventComponent, ProteinComponent
from openff.units import unit

protein = ProteinComponent.from_pdb_file('inputs/tyk2_protein.pdb')

# Note: the distance from the solute to add water is not defined here but in the
# the relevant RBFE solver method
solvent = SolventComponent(positive_ion='Na', negative_ion='Cl',
                           neutralize=True, ion_concentration=0.15*unit.molar)

In [ ]:
# Extract the relevant edge for the lig_ejm_31 -> lig_ejm_47 transform in the radial graph
ejm_31_to_ejm_47 = [edge for edge in network.edges if edge.componentB.name == "lig_ejm_47"][0]

ejm_31_to_ejm_47

In [ ]:
# Let's create the four ChemicalSystems
from openfe import ChemicalSystem

ejm_31_complex = ChemicalSystem({'ligand': ejm_31_to_ejm_47.componentA,
                                  'solvent': solvent,
                                  'protein': protein,})
ejm_31_solvent = ChemicalSystem({'ligand': ejm_31_to_ejm_47.componentA,
                                  'solvent': solvent,})

ejm_47_complex = ChemicalSystem({'ligand': ejm_31_to_ejm_47.componentB,
                                 'solvent': solvent,
                                 'protein': protein,})
ejm_47_solvent = ChemicalSystem({'ligand': ejm_31_to_ejm_47.componentB,
                                 'solvent': solvent,})

### Defining the RBFE simulation settings

There are various different parameters which can be set to determine
how the RBFE simulation will take place. To allow for maximum user flexibility, these are defined as a series of settings objects which control the following::

1. `protocol_repeats`: The number of completely independent repeats of the entire sampling process.

2. `simulation_settings`: Parameters controling the simulation plan and the alchemical sampler, including the number of minimization steps, lengths of equilibration and production runs, the sampler method (e.g. Hamiltonian replica exchange, `repex`), and the time interval at which to perform an analysis of the free energies.

3. `output_settings`: Simulation output control settings, including the frequency to write a checkpoint file, the selection string for which part of the system to write coordinates for, and the paths to the trajectory and output structure storage files.

4. `alchemical_settings`: Parameters controlling the creation of the hybrid topology system. This includes various parameters ranging from softcore parameters, through to whether or not to apply an explicit charge correction for systems with net charge changes.

5. `engine_settings`: Parameters determining how the OpenMM engine will execute the simulation. This controls the compute platform which will be used to carry out the simulation.

6. `integrator_settings`: Parameters controlling the `LangevinSplittingDynamicsMove` integrator used for simulation.

7. `lambda_settings`: Lambda protocol settings, including number of lambda windows and lambda functions.

8. `forcefield_settings`: Parameters to set up the force field with OpenMM Force Fields, including the general forcefields, the small molecule forcefield, the nonbonded method, and the nonbonded cutoff.

9. `thermo_settings`: Settings for thermodynamic parameters, such as the temperature and the pressure of the system.

10. `solvation_settings`: Settings for solvating the system, including the solvent model and the solvent padding.

11. `partial_charge_settings`: Settings for assigning partial charges to small molecules, including the partial charge method (e.g. `am1bcc`) and the openff toolkit backend (e.g. `ambertools` or `openeye`).


By default these settings use the values which are considered appropriate for RBFE calculations, however these are very system dependent and may not always be suitable for every case. A judicious choice of settings is always advised.

The `RelativeHybridTopologyProtocol` class can directly populate the above set of default
settings through its `default_settings` method.

In [ ]:
from openfe.protocols.openmm_rfe import RelativeHybridTopologyProtocol
from pprint import pprint

rbfe_settings = RelativeHybridTopologyProtocol.default_settings()
pprint(rbfe_settings.dict())

In [ ]:
from pprint import pp
from openff.units import unit

# Parameters can be overriden after creation
# In this case, we'll reduce the equilibration length to 0.01 * nanosecond
# and the production to 0.05 * nanosecond in order to reduce the costs of
# running this notebook (in practice values of 2 and 5 nanoseconds
# respectively would be most appropriate)

rbfe_settings.simulation_settings.equilibration_length = 10 * unit.picosecond
rbfe_settings.simulation_settings.production_length = 50 * unit.picosecond
rbfe_settings.engine_settings.gpu_device_index = None  # default is to require CUDA, None allows for falling back to OpenCL or CPU

pprint(rbfe_settings.simulation_settings.dict())

### Creating the RelativeLigandTransform Protocol

With the Settings inspected and adjusted, we can provide these to the Protocol.
This Protocol defines the procedure to estimate a free energy difference between two chemical systems,
with the details of the two end states yet to be defined.

In [ ]:
# Create RBFE Protocol class
rbfe_transform = RelativeHybridTopologyProtocol(
    settings=rbfe_settings
)

### Creating the Protocol DAGs

With the **method** we will be applying (the `Protocol`) defined, we can move onto applying to the alchemical transformation of interest.

The `Protocol.create()` method requires as input:
- the two `ChemicalSystem` objects defining either end of the alchemical transformation
- a mapping between the two systems, as a dict

This creates a directed-acyclic-graph (DAG) of computational tasks necessary for creating an estimate of the free energy difference between the two chemical systems.

As previously detailed, we create two sets of simulations, defining
both the complex and solvent transformations.

In [ ]:
print(ejm_31_to_ejm_47)

In [ ]:
complex_dag = rbfe_transform.create(
    stateA=ejm_31_complex, stateB=ejm_47_complex,
    mapping=ejm_31_to_ejm_47,
)

solvent_dag = rbfe_transform.create(
    stateA=ejm_31_solvent, stateB=ejm_47_solvent,
    mapping=ejm_31_to_ejm_47,
)

The individual pieces of computational work are called `Unit`s.  In this particular Protocol, the Units defined are three independent repeats of the alchemical transformation.

For other Protocols, for example non-equilibrium sampling routines, there might be dependencies between the tasks.

In [ ]:
list(complex_dag.protocol_units)

### Simulating the RelativeLigandTransforms

Individual Units can then be executed by calling the `.execute()` method.

In the first instance we do a dry-run (which does everything but
starting the simulation) to make sure that the
hybrid openmm system can be constructed without any issues.
Note: A successful call to `.run()` will return an empty Dictionary.

In [ ]:
# complex dry-run
complex_unit = list(complex_dag.protocol_units)[0]

complex_unit.run(dry=True, verbose=True)

In [ ]:
# solvent dry-run
solvent_unit = list(solvent_dag.protocol_units)[0]

solvent_unit.run(dry=True, verbose=True)

### Executing the simulations.

The complex and solvent DAGs each contain many invdividual jobs. For convenience we can execute them sequentially in this notebook using the `gufe.protocols.execute` function.

In a more realistic (expansive) situation we would farm off the individual jobs to a HPC cluster or cloud compute service so they could be executed in parallel.

**Note: we use the `shared_basedir` and `scratch_basedir` argument of `execute_DAG` in order to set the directory where the simulation files are written to**

In [ ]:
from gufe.protocols import execute_DAG
import pathlib

In [ ]:
# Finally we can run the simulations
complex_path = pathlib.Path('./complex')
complex_path.mkdir()

# First the complex transformation
complex_dag_results = execute_DAG(complex_dag, scratch_basedir=complex_path, shared_basedir=complex_path)

In [ ]:
# Next the solvent state transformation
solvent_path = pathlib.Path('./solvent')
solvent_path.mkdir()

solvent_dag_results = execute_DAG(solvent_dag, scratch_basedir=solvent_path, shared_basedir=solvent_path)

## 4. Analysis

Finally now that we've run our simulations, let's go ahead and gather the free
energies for both phases.

This can be achieved by passing the results of executing the DAGs calling the `gather()` methods of `RelativeLigandTransform`.
This takes a **list** of completed DAG results, catering for when simulations have been extended.

In [ ]:
# Get the complex and solvent results
complex_results = rbfe_transform.gather([complex_dag_results])
solvent_results = rbfe_transform.gather([solvent_dag_results])

print(f"Complex dG: {complex_results.get_estimate()}, err {complex_results.get_uncertainty()}")
print(f"Solvent dG: {solvent_results.get_estimate()}, err {solvent_results.get_uncertainty()}")